In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import model_selection, preprocessing
import xgboost as xgb
from datetime import timedelta
import datetime
import math
import time

In [2]:
cd /Users/williamzhou/Downloads/RussianHousing-master

/Users/williamzhou/Downloads/RussianHousing-master


In [3]:
train=pd.read_csv('./data/processed/Clean0517/train_clean_shu_0517.csv')
test = pd.read_csv('./data/processed/Clean0517/test_clean_shu_0517.csv')
macro = pd.read_csv('./data/raw/macro.csv')
# latlon = pd.read_csv('./data/external/sub_area_lon_lat.csv')
feature_importance = pd.read_csv('/Users/williamzhou/Documents/github/RussianHousing/feature engineering/feature_importance.csv')

train['timestamp'] = pd.to_datetime(train['timestamp'])
macro['timestamp'] = pd.to_datetime(macro['timestamp'])
test['timestamp'] = pd.to_datetime(test['timestamp'])

train['label']='train'
test['label']='test'

print ('train shape',train.shape)
print ('test shape',test.shape)
print ('feature_importance shape', feature_importance.shape)

('train shape', (30471, 300))
('test shape', (7662, 297))
('feature_importance shape', (276, 3))


# Merge with Macro data

In [4]:
time_laps_dict ={
#                     'usdrub':0#,
                 'micex_cbi_tr':0,
                 'micex_rgbi_tr':0#,
#                  'deposits_value':30
#                  'ppi':90,
#                  'cpi':300
                }

def merge_macro_feature(dicts,macro,df):
    
    for item in dicts:
        macro_timeshift=macro.copy()
        macro_timeshift['timestamp']=macro.timestamp+timedelta(days=dicts[item])
        df = pd.merge(df,macro_timeshift[['timestamp',item]],on='timestamp',how='left')
    print('data shape:',df.shape)
    return(df)

train =merge_macro_feature(time_laps_dict,macro,train)
test=merge_macro_feature(time_laps_dict,macro,test)

('data shape:', (30471, 302))
('data shape:', (7662, 299))


In [5]:
#### Group frequent area
freq_area = np.array(train.loc[:, 'sub_area'].value_counts()[:2].index)
train.loc[~train['sub_area'].isin(freq_area), 'sub_area'] = 'other'
test.loc[~test['sub_area'].isin(freq_area), 'sub_area'] = 'other'
print ('subarea are {}'.format(train.sub_area.unique()))

subarea are ['other' 'Poselenie Sosenskoe']


In [6]:
train.loc[train['full_sq'].isnull(),'full_sq']=train['full_sq'].median()
test.loc[test['full_sq'].isnull(),'full_sq']=test['full_sq'].median()
train['price_full_sq']=train['price_doc']/train['full_sq']
train['price_full_sq']=train['price_full_sq'].astype('int64')

# train= median_price_sqm_by_ID('ID_metro',train)
# test= median_price_sqm_by_ID('ID_metro',test)
print ('train shape',train.shape)
print ('test shape',test.shape)

('train shape', (30471, 303))
('test shape', (7662, 299))


In [7]:
y_train=train[['price_full_sq','sub_area']]
x_train= train.drop(['price_doc','price_full_sq'],axis=1)
test_id = test[['id','sub_area']]
x_test = test.drop(['id'],axis=1)

print('x_train shape:',x_train.shape)
print('y_train shape:',y_train.shape)
print('x_test shape:',x_test.shape)
print('test_id shape:',test_id.shape)


('x_train shape:', (30471, 301))
('y_train shape:', (30471, 2))
('x_test shape:', (7662, 298))
('test_id shape:', (7662, 2))


In [8]:
df_all = pd.concat([x_train,x_test])

In [9]:
x_train=df_all.loc[df_all.label=='train',:]
x_train.shape

(30471, 301)

In [10]:
IDs = ['ID_big_road1','ID_big_road2',
        'ID_bus_terminal','ID_metro',
       'ID_railroad_station_avto',
       'ID_railroad_terminal']

def to_object(ID,df):
    df[ID] = df[ID].astype(object)
    return(df)

df_all=to_object(IDs,df_all)

    
month_year = (df_all.timestamp.dt.month + df_all.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (df_all.timestamp.dt.weekofyear + df_all.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

for c in df_all.columns:
    if df_all[c].dtype == 'object' and c!='label' and c!='sub_area':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_all[c].values)) 
        df_all[c] = lbl.transform(list(df_all[c].values))

df_all=df_all.drop(['timestamp'],axis =1 )

x_train = df_all.loc[df_all.label=='train',:]
x_test = df_all.loc[df_all.label=='test',:]

print('x_train shape',x_train.shape)
print('x_test shape',x_test.shape)
print('y_train shape',y_train.shape)

('x_train shape', (30471, 302))
('x_test shape', (7662, 302))
('y_train shape', (30471, 2))


In [11]:
final_features = pd.read_csv('./cv_output/final_feature.csv')
train_feature = list(final_features.iloc[:,0])
print('Training XGBoost with %d features'%len(train_feature))

Training XGBoost with 40 features


In [12]:
subareas=x_train.sub_area.unique()
print('Subareas has %d areas'%len(subareas))


Subareas has 2 areas


In [13]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}

In [14]:
def median_missing_imputation(df):
    for cols in df.columns:
            df.loc[pd.isnull(df[cols]),cols] = np.median(df.loc[pd.isnull(df[cols])==False,cols])

    return(df)

In [15]:
%%time
sub=pd.DataFrame()
for area in subareas:
# for area in ['Strogino']:
    x_train_subset = x_train.loc[x_train.sub_area==area,train_feature].reset_index(drop=True)
    x_test_subset = x_test.loc[x_test.sub_area==area,train_feature].reset_index(drop=True)
    y_train_subset = y_train.loc[x_train.sub_area==area,'price_full_sq'].reset_index(drop=True)
    test_id_subset =test_id.loc[test_id.sub_area==area,'id'].reset_index(drop=True)
    
#     missing imputation using on subarea median
#     x_train_subset = median_missing_imputation(x_train_subset)
#     x_test_subset = median_missing_imputation(x_test_subset)
    print('Finish missing imputation.')
    dtrain_subset = xgb.DMatrix(x_train_subset, y_train_subset)
    dtest_subset =  xgb.DMatrix(x_test_subset)
    print('{} x_train_subset shape {}'.format(area,x_train_subset.shape))
    print('{} y_test_subset shape {}'.format(area,y_train_subset.shape))
    print('{} x_test_subset shape {}'.format(area,x_test_subset.shape))
    print('{} test_id_subset shape {}'.format(area,test_id_subset.shape))
    print('Ready! Start training {}......'.format(area))
    
    cv_output = xgb.cv(xgb_params, dtrain_subset, 
                   num_boost_round=1000, 
                   early_stopping_rounds=20,
                   verbose_eval=50, show_stdv=False)
    cv_output.to_csv('./train_by_subarea/{}.csv'.format(area))
    print('{}.csv successfully saved!'.format(area))
    num_boost_rounds = len(cv_output)
    print('Best number of iteration for {} model: {})'.format(area,num_boost_rounds))
    model = xgb.train(dict(xgb_params, silent=0), dtrain_subset, num_boost_round= num_boost_rounds)
    print('Finish XGBoost training')
    print('Predicting........')
    y_predict = model.predict(dtest_subset)
    y_predic_all_sq = (y_predict)*x_test_subset['full_sq']
#     print(y_predic_all_sq)
#     print(test_id.id[test_id.sub_area==area])
    df_sub = pd.DataFrame({'id':test_id_subset , 'price_doc': y_predic_all_sq})
    print([x for x in df_sub])
    sub = pd.concat([sub,df_sub])
    print('Successfully trained {}'.format(area))
    print('----------------------------------------------------------')
    print('----------------------------------------------------------')
print('The prediction dataset should have 7662 rows of data, your dataset has {} rows of data'.format(len(sub)))

sub.sort_values(by='id').reset_index(drop=True).to_csv('./train_by_subarea/sub_{}.csv'.format(time.ctime()),index=False)
print('Successfully saved submission file! ')

Finish missing imputation.
other x_train_subset shape (28695, 40)
other y_test_subset shape (28695,)
other x_test_subset shape (7365, 40)
other test_id_subset shape (7365,)
Ready! Start training other......
[0]	train-rmse:140001	test-rmse:140009
[50]	train-rmse:43521.4	test-rmse:45230.2
[100]	train-rmse:39923.7	test-rmse:42714.3
[150]	train-rmse:38309.1	test-rmse:42143.6
[200]	train-rmse:37024.7	test-rmse:41857.6
[250]	train-rmse:35942.8	test-rmse:41678.9
[300]	train-rmse:35035.8	test-rmse:41594.6
[350]	train-rmse:34166	test-rmse:41524.9
[400]	train-rmse:33324.1	test-rmse:41505.5
other.csv successfully saved!
Best number of iteration for other model: 393)
Finish XGBoost training
Predicting........
['id', 'price_doc']
Successfully trained other
----------------------------------------------------------
----------------------------------------------------------
Finish missing imputation.
Poselenie Sosenskoe x_train_subset shape (1776, 40)
Poselenie Sosenskoe y_test_subset shape (1776,)
P